In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Loading the data

In [2]:

#!pip install pymysql
import pandas as pd

In [3]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Get the reviews

In [ ]:
df = '/content/drive/MyDrive/Copy of database.sqlite'


In [ ]:
import sqlite3


In [ ]:
sqlite_file = df
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''SELECT content,genre FROM content JOIN genres ON content.reviewid = genres.reviewid ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]

In [ ]:
df

,content,genre
0,"“Trip-hop” eventually became a ’90s punchline,...",electronic
1,"Eight years, five albums, and two EPs in, the ...",metal
2,Minneapolis’ Uranium Club seem to revel in bei...,rock
3,Kleenex began with a crash. It transpired one ...,rock
4,It is impossible to consider a given release b...,electronic
...,...,...
22685,"Out of Tune is a Steve Martin album. Yes, I'l...",rock
22686,"Well, kids, I just went back and re-read my re...",experimental
22687,"Well, kids, I just went back and re-read my re...",metal
22688,"Well, kids, I just went back and re-read my re...",rock


In [ ]:
df['input_text'] = df['genre'] + ": " + df['content']

In [ ]:
df.to_csv('genre_music_final.csv')

PreProcessing.

In [5]:
#df_reviews = pd.read_csv('/content/drive/MyDrive/music_review_dataset_final.csv')
#df_reviews = pd.read_csv('/content/drive/MyDrive/music_album_reviews.csv')
df_reviews = pd.read_csv('/content/drive/MyDrive/genre_music_final.csv')

In [ ]:
df_reviews

,Unnamed: 0,content,genre,input_text
0,0,"“Trip-hop” eventually became a ’90s punchline,...",electronic,electronic: “Trip-hop” eventually became a ’90...
1,1,"Eight years, five albums, and two EPs in, the ...",metal,"metal: Eight years, five albums, and two EPs i..."
2,2,Minneapolis’ Uranium Club seem to revel in bei...,rock,rock: Minneapolis’ Uranium Club seem to revel ...
3,3,Kleenex began with a crash. It transpired one ...,rock,rock: Kleenex began with a crash. It transpire...
4,4,It is impossible to consider a given release b...,electronic,electronic: It is impossible to consider a giv...
...,...,...,...,...
22685,22685,"Out of Tune is a Steve Martin album. Yes, I'l...",rock,rock: Out of Tune is a Steve Martin album. Ye...
22686,22686,"Well, kids, I just went back and re-read my re...",experimental,"experimental: Well, kids, I just went back and..."
22687,22687,"Well, kids, I just went back and re-read my re...",metal,"metal: Well, kids, I just went back and re-rea..."
22688,22688,"Well, kids, I just went back and re-read my re...",rock,"rock: Well, kids, I just went back and re-read..."


In [6]:
df_reviews = df_reviews.drop(['Unnamed: 0'],axis=1)
#df_reviews = df_reviews.drop(['Rating'],axis=1).rename({'Review':'content'},axis=1)

In [7]:
df_reviews = df_reviews.drop(['content','genre'],axis=1).rename({'input_text':'content'},axis=1)


In [8]:
df_reviews

,content
0,electronic: “Trip-hop” eventually became a ’90...
1,"metal: Eight years, five albums, and two EPs i..."
2,rock: Minneapolis’ Uranium Club seem to revel ...
3,rock: Kleenex began with a crash. It transpire...
4,electronic: It is impossible to consider a giv...
...,...
22685,rock: Out of Tune is a Steve Martin album. Ye...
22686,"experimental: Well, kids, I just went back and..."
22687,"metal: Well, kids, I just went back and re-rea..."
22688,"rock: Well, kids, I just went back and re-read..."


In [9]:
import re

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download


def clean_reviews(df, review_col='content'):
    """
    Clean the reviews in a pandas DataFrame by lowercasing and replacing newlines,
    without lemmatization and stopwords removal, and without removing punctuation.

    Args:
    df (pd.DataFrame): The input DataFrame containing reviews.
    review_col (str): The name of the column containing the reviews.

    Returns:
    list: A list of cleaned reviews, each followed by an end-of-document token.
    """
    eod_token = "<|endoftext|>"
    processed_reviews = []

    for review in df[review_col]:
        if isinstance(review, str):
            # Lowercase the review text
            clean_review = review.lower()
            # Replace newlines with spaces
            clean_review = clean_review.replace("\n", " ")
            # Append end-of-document token
            processed_reviews.append(f"{clean_review} {eod_token}")

    return processed_reviews

In [10]:
df_reviews['content']

0        electronic: “Trip-hop” eventually became a ’90...
1        metal: Eight years, five albums, and two EPs i...
2        rock: Minneapolis’ Uranium Club seem to revel ...
3        rock: Kleenex began with a crash. It transpire...
4        electronic: It is impossible to consider a giv...
                               ...                        
22685    rock: Out of Tune is a Steve Martin album.  Ye...
22686    experimental: Well, kids, I just went back and...
22687    metal: Well, kids, I just went back and re-rea...
22688    rock: Well, kids, I just went back and re-read...
22689                                                  NaN
Name: content, Length: 22690, dtype: object

In [11]:
string = [i for i in df_reviews['content']]
'jax' in string

False

In [12]:
# Apply the function to each row in the 'reviews' column

list_of_reviews = clean_reviews(df_reviews)


In [13]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from torch.utils.data import Dataset, DataLoader

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
class GPT2Dataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

In [ ]:
# cleaned_reviews_text = "\n".join(text)
tokenizer.pad_token = tokenizer.eos_token



input_ids = []
attention_masks = []

for review in list_of_reviews:
    encodings = tokenizer(review, return_tensors='pt', padding='max_length', truncation=True, max_length=120)
    input_ids.append(encodings['input_ids'].squeeze().tolist())
    attention_masks.append(encodings['attention_mask'].squeeze().tolist())

# Convert lists to tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)


In [ ]:
with open('tokenized_reviews.pt', 'wb') as f:
    torch.save(encodings, f, pickle_protocol=4)

In [ ]:
import torch


In [ ]:
#tokens = torch.load('/content/drive/MyDrive/MusicReview/tokens.pt')
tokens = torch.load('/content/drive/MyDrive/tokens_short.pt')

## Setting up x and y pairs.

In [ ]:
from sklearn.model_selection import train_test_split

train_size = 0.8
train_indices, val_indices = train_test_split(range(len(input_ids)), train_size=train_size, random_state=42)

train_encodings = {
    'input_ids': [input_ids[i] for i in train_indices],
    'attention_mask': [attention_masks[i] for i in train_indices]
}

val_encodings = {
    'input_ids': [input_ids[i] for i in val_indices],
    'attention_mask': [attention_masks[i] for i in val_indices]
}

# Create datasets
train_dataset = GPT2Dataset(train_encodings)
val_dataset = GPT2Dataset(val_encodings)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)


## Finetuning

In [ ]:
from transformers import GPT2LMHeadModel,GPT2Config


In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
# # Define the model configuration
# config = GPT2Config()

# # Instantiate the model with random weights using the configuration
# model = GPT2LMHeadModel(config)


In [ ]:
# Freeze all parameters first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last linear layer
model.lm_head.weight.requires_grad = True

In [ ]:
torch.cuda.empty_cache()

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
for name, param in model.named_parameters():
    print(f"{name} is on {param.device}")

transformer.wte.weight is on cuda:0
transformer.wpe.weight is on cuda:0
transformer.h.0.ln_1.weight is on cuda:0
transformer.h.0.ln_1.bias is on cuda:0
transformer.h.0.attn.c_attn.weight is on cuda:0
transformer.h.0.attn.c_attn.bias is on cuda:0
transformer.h.0.attn.c_proj.weight is on cuda:0
transformer.h.0.attn.c_proj.bias is on cuda:0
transformer.h.0.ln_2.weight is on cuda:0
transformer.h.0.ln_2.bias is on cuda:0
transformer.h.0.mlp.c_fc.weight is on cuda:0
transformer.h.0.mlp.c_fc.bias is on cuda:0
transformer.h.0.mlp.c_proj.weight is on cuda:0
transformer.h.0.mlp.c_proj.bias is on cuda:0
transformer.h.1.ln_1.weight is on cuda:0
transformer.h.1.ln_1.bias is on cuda:0
transformer.h.1.attn.c_attn.weight is on cuda:0
transformer.h.1.attn.c_attn.bias is on cuda:0
transformer.h.1.attn.c_proj.weight is on cuda:0
transformer.h.1.attn.c_proj.bias is on cuda:0
transformer.h.1.ln_2.weight is on cuda:0
transformer.h.1.ln_2.bias is on cuda:0
transformer.h.1.mlp.c_fc.weight is on cuda:0
transfo

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

total_parameters = total_params


Total Parameters: 124439808
Trainable Parameters: 38597376


In [ ]:
from tqdm import tqdm
import torch


import torch
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Assuming the following have already been defined:
# - model
# - device
# - optimizer
# - train_dataloader
# - val_dataloader

model.to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-2)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

epochs = 10
for epoch in range(epochs):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    count = 0

    # Training loop
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        count += 1

        inputs, mask = batch['input_ids'], batch['attention_mask']
        inputs = inputs.to(device)
        mask = mask.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=inputs, labels=inputs, attention_mask=mask)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        tr_loss += loss.item()
        nb_tr_examples += inputs.size(0)
        nb_tr_steps += 1

    # Calculate average training loss
    avg_train_loss = tr_loss / nb_tr_steps

    # Validation loop
    model.eval()
    val_loss = 0
    nb_val_examples, nb_val_steps = 0, 0

    with torch.no_grad():
        for step, batch in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
            inputs, mask = batch['input_ids'], batch['attention_mask']
            inputs = inputs.to(device)
            mask = mask.to(device)

            outputs = model(input_ids=inputs, labels=inputs, attention_mask=mask)
            loss = outputs.loss

            val_loss += loss.item()
            nb_val_examples += inputs.size(0)
            nb_val_steps += 1

    # Calculate average validation loss
    avg_val_loss = val_loss / nb_val_steps
    scheduler.step(avg_val_loss)

    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Training completed. Average training loss: {avg_train_loss:.4f}')
    print(f'Validation completed. Average validation loss: {avg_val_loss:.4f}')
    torch.cuda.empty_cache()



/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
  0%|          | 0/254 [00:00<?, ?it/s]<ipython-input-87-9988673b3ff8>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 64/64 [00:22<00:00,  2.88it/s]


Epoch 1/10
Training completed. Average training loss: 5.1642
Validation completed. Average validation loss: 4.3197


100%|██████████| 64/64 [00:22<00:00,  2.88it/s]


Epoch 2/10
Training completed. Average training loss: 4.2043
Validation completed. Average validation loss: 4.2721


100%|██████████| 64/64 [00:22<00:00,  2.88it/s]


...

Epoch 8/10
Training completed. Average training loss: 2.8893
Validation completed. Average validation loss: 3.9878


100%|██████████| 64/64 [00:22<00:00,  2.89it/s]


Epoch 9/10
Training completed. Average training loss: 2.8395
Validation completed. Average validation loss: 3.9757


100%|██████████| 64/64 [00:22<00:00,  2.89it/s]


Epoch 10/10
Training completed. Average training loss: 2.7999
Validation completed. Average validation loss: 3.9745


In [ ]:
torch.save(model.state_dict(), 'model_final_best_two.pt')


## Loading model and inference

In [22]:
state_dict = torch.load('/content/drive/MyDrive/Copy of model_final_best_two.pt')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [24]:
model.eval()

# Initial text for generation
initial_text = "hip hop:"

# Tokenize the input text
encoded_input = tokenizer(initial_text, return_tensors='pt')
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)


# Generate text
generated_text_samples = model.generate(
    input_ids,
    attention_mask=attention_mask,

    max_length=100,
    do_sample=True,# Enable sampling to use top_p
    top_p=0.92,             # Set the cumulative probability threshold
    top_k=0,                # It's recommended to set top_k to 0 when using top_p
    temperature=0.7,        # Set the temperature for sampling
    repetition_penalty=1.2, # Penalty for repetition to avoid loops
    num_return_sequences=5  # Number of sequences to generate
)

# Decode and print the generated text
for i, sample in enumerate(generated_text_samples):
    print(f"Generated Text {i+1}: {tokenizer.decode(sample, skip_special_tokens=True)}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: hip hop: i hate you, bitch.  "i won't stop" is a lyric that usually means the absence of desire or jealousy-- and it's true; after all this time spent making music for people who never really cared about what they heard in public school radio mulatu vibes were like watching an old acquaintance straddle with his deadringer before he died while listening to sped-up rap songs on one end (to get pissed off at how fun some adults are getting when kids
Generated Text 2: hip hop: there's a name for the album, courtesy of dirty projectors. it follows four years of quirky pop-savantia that resulted in one record from six people creating completely different songs with no regard to who was making them-- which is sorta odd given how dorky and brooding downhearted hipness often seem on an idea level at best.  but instead holy fuck! take la roux; you're gonna miss me here anyway. so let's get this out
Generated Text 3: hip hop: so i can't really count it, but this is the only song